In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [4]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [5]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [6]:
sys.meta_path.append(NotebookFinder())

In [7]:
import myprepdata

importing Jupyter notebook from myprepdata.ipynb


In [8]:
import encoder

importing Jupyter notebook from encoder.ipynb


In [9]:
import decoder

importing Jupyter notebook from decoder.ipynb


In [10]:
dataset_loader = myprepdata.prep()

[  8.89466166e-01   2.06377721e+00   6.17529333e-01   3.18206835e+00
   7.67453134e-01  -1.99592626e+00  -1.48355320e-01  -1.96766481e-01
  -3.16980266e-04   2.14467272e-01  -4.14004624e-01   3.57001126e-01
  -1.75457168e+00  -1.39167953e+00   2.63751793e+00   1.19258475e+00
   6.61663294e-01  -3.63710284e-01   1.10500908e+00  -1.90649366e+00
  -1.27631116e+00   7.06363976e-01   2.16604018e+00   7.20149457e-01
   1.19366109e+00   3.15650672e-01  -3.05436397e+00   7.40569711e-01
   8.19141865e-01   1.18011284e+00   4.69861925e-01   8.27771947e-02
   1.12338221e+00   2.47764182e+00   3.43266904e-01  -2.68325806e+00
  -1.40979004e+00  -1.32848775e+00   7.03199029e-01   9.64148343e-02
   3.22712660e-01  -2.13381246e-01  -1.01692128e+00   1.25775146e+00
   1.02071427e-01  -1.11394691e+00  -2.41053867e+00  -1.19636559e+00
   4.32757884e-01   1.38155115e+00   1.13593802e-01  -1.06976998e+00
   1.57687080e+00   1.43941748e+00   2.68036485e-01   1.28006709e+00
   1.21146309e+00   6.14654839e-01

In [11]:
newline = dataset_loader.prepare_line()[0]
newline

'Res@@ um@@ ption of the session'

In [12]:
deline = dataset_loader.de_prepare_line()[0]
deline

'Wiederaufnahme der Sitzungsperiode'

In [13]:
dataset_loader.get_wvmodel().wv["Res@@"]

array([-0.67258787, -4.15615225, -1.72683489,  1.6745466 , -0.66765279,
        1.20479822, -0.54290372, -0.17763264, -6.63635206, -3.36331749,
       -1.20754302, -4.75114679,  3.3733356 , -2.00328326,  1.61597621,
       -2.5903039 ,  3.89082551,  2.05730581, -0.37433237, -1.15445471,
        0.9606185 ,  4.23989391,  1.05038905, -1.0970273 , -3.27382827,
        1.57637691, -4.19764471, -2.15746975,  2.48696518, -0.61238277,
       -1.74696052, -0.37781578,  0.34524947,  2.54945087, -1.64452887,
        0.57680875, -0.14833695,  1.02612078, -0.26212877, -0.51345897,
       -1.39085841,  0.09278773, -1.48713863, -0.46643406,  7.02497864,
        2.74286199, -0.83628732, -1.48309898, -0.32239187, -3.01059842,
        1.07048142, -3.00496078,  0.34531841,  3.417485  ,  2.11597872,
        0.14785768,  0.43224403,  0.61093956, -2.73394179, -1.91496217,
       -2.31602764,  2.11022353,  3.03709865,  0.61163723,  0.87978035,
       -1.34368658, -4.88440132, -1.7299546 , -0.21339908, -1.17

In [14]:
len(newline.split(' '))

6

In [15]:
newline.split(' ')[0]

'Res@@'

In [16]:
padding_line = np.full((1, 484, 100), 0, dtype="float32")
de_padding_line = np.full((1, 3, 100), 0, dtype="float32")
i = 0
#padding_line

In [17]:
while(i < len(newline.split(' '))):
    padding_line[0][i] = dataset_loader.get_wvmodel().wv[newline.split(' ')[i]]
    i = i + 1
i = 0

while(i < len(deline.split(' '))):
    de_padding_line[0][i] = dataset_loader.de_get_wvmodel().wv[deline.split(' ')[i]]
    i = i + 1

In [18]:
de_padding_line[0][2]

array([-1.16796255,  4.08470106, -1.14912891,  0.16463454, -0.60790056,
        0.98632014,  0.03656695, -1.04373217,  4.78645897,  3.07820916,
       -0.23398891,  2.22593546, -0.22403085,  0.36990732,  4.49744558,
        0.15719461, -1.70705783, -1.05890298,  2.06401539, -4.90328693,
       -1.15247893,  1.36518836, -0.30293822,  3.44166207,  0.95330393,
       -0.92819381, -0.33055443, -1.3885417 , -0.45812017,  1.86859751,
       -0.60691816, -0.73033249,  2.29482031, -1.34969199, -1.00371766,
       -3.21881509, -0.60357022,  0.05063005,  0.37750426,  1.24002087,
        0.33714008, -1.77580595, -0.61027128, -0.23812634, -1.28080368,
       -3.05595374, -2.05308676, -0.70262378,  0.17834105, -0.98828769,
       -1.62905455,  1.10368884,  1.40606165,  2.52097225,  0.14530618,
       -0.47233748, -1.1117965 , -1.03726923,  2.86678886,  2.92978334,
       -2.53581524,  0.39890549, -2.79667115, -0.61639524,  1.67488325,
       -1.26576602, -0.78218991, -0.09562778,  0.27236822, -0.84

In [19]:

with tf.Graph().as_default(): #set default graph
    encoder1 = encoder.naive_encoder(layers=1, directions =1, maxv=484, hiddensizes=100, bach_s=1)

    decoder1 = decoder.attention_decoder(layers = 1, directions = 1, hiddensize = 100, input_length = 484, 
                     output_v = de_padding_line, output_length = len(deline.split(' ')), teacher_forcing = False)

    #workers, _ = tf.test.create_local_cluster(1, 0) #python 3.6 <
    #worker = workers[0] #python 3.6 < 


    with tf.Session() as sess:
            
        encoder1.hidden_init(sess)
        decoder1.hidden_init(sess)
        encoder1.train(sess,padding_line)
        
        print(np.shape(encoder1.getoutput(sess,padding_line)))
        print(np.shape(encoder1.getstate(sess,padding_line)))
        
        en_out = encoder1.getoutput(sess,padding_line)
        en_state = encoder1.getstate(sess,padding_line)
        
        print("decoder1forword:", np.shape(decoder1.forword(sess,en_out[0][0],en_state[0])))

        decoder1.calcost(sess,de_padding_line[0],en_out[0][0],en_state[0])
        #decoder1.training(sess,en_out[0][0],en_state[0])


hidden =  [array([[  7.37671690e-27,  -8.27621786e-28,   4.01945893e-26,
          2.34061841e-26,  -3.11953812e-27,  -1.66642706e-26,
         -1.34365433e-25,   1.84866857e-26,   3.16183185e-26,
          1.07770368e-25,  -8.46192727e-26,   2.98422167e-26,
          1.26254698e-25,  -5.47110537e-26,  -2.21898733e-25,
         -1.95382160e-25,   1.15289901e-25,   7.82334129e-26,
         -2.20178936e-27,  -4.45920883e-26,  -1.21213606e-25,
          5.89048539e-26,  -1.27377161e-25,  -1.35287796e-25,
          4.57606008e-26,  -1.24194317e-25,   9.81226856e-26,
         -1.51041065e-25,  -2.99415565e-27,  -2.34971681e-26,
          1.36964261e-25,  -7.61036548e-26,  -2.52193562e-26,
         -1.54978467e-25,   1.02527605e-25,  -3.58128923e-25,
         -1.69437375e-25,   1.15584948e-25,  -2.71511841e-26,
          6.57874373e-26,  -2.19508158e-26,  -2.39391686e-27,
          7.11281735e-26,   4.35291599e-26,   8.17944296e-26,
          7.52072192e-26,  -8.65555750e-26,  -1.95063411e-2

In [20]:
#import portpicker